In [ ]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [ ]:
import geemap
import ee
Map=geemap.Map()
Map

In [ ]:
# 武汉市
roi = ee.FeatureCollection('users/311605001111/ChinaCity').filter(ee.Filter.eq('市','武汉市'))
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,8)

In [ ]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

visParams = {
    'bands': ['B5', 'B6', 'B4'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

# AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI") 
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    return image.addBands(ndvi_mndwi).addBands(ndvi).addBands(mndwi).addBands(awei)

def area_after_clip(image):
    image_area = image.clip(roi).select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': image.geometry(),
        'scale': 1000,
        'maxPixels': 1e14
    })
    return image.set({'area': areas.get('B2')})

In [ ]:
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr) \
    .map(water_index)
Map.addLayer(landsat_images.mosaic(),visParams,'landsat image')

# 筛选得出采样影像

In [ ]:
image_area = landsat_images.mosaic().select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
areas = image_area.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': roi,
    'scale': 1000,
    'maxPixels': 1e14
})

# 筛选影像，影像覆盖研究区的范围 > 研究区的1/10
sample_image = landsat_images.map(area_after_clip).filter(ee.Filter.gt('area',ee.Number(areas.get('B2')).divide(10)))
dates = sample_image.aggregate_array('system:time_start').sort().map(lambda d: ee.Date(d).format('YYYY-MM-dd'))
print(sample_image.size().getInfo())
print(dates.getInfo())

# 多水体频率数据集的类别范围确定

In [ ]:
# ndvi_mndwi水体频率
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
water_layer = landsat_images.map(lambda i : i.select('ndvi_mndwi').lt(0)).sum()
validPixel_layer = landsat_images.count().select('B3').clip(roi).rename('count')
index_waterfrequency = water_layer.select('ndvi_mndwi').divide(validPixel_layer.select('count')).select('ndvi_mndwi').rename('frequency')
index_permanent = index_waterfrequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
index_season= ee.ImageCollection([index_waterfrequency.gte(0.25),index_waterfrequency.lt(0.75)]).sum().eq(2).rename('waterclass')
index_waterclass = ee.ImageCollection([index_permanent,index_season,basemap]).sum()
# JRC
JRC_2018 = ee.Image("JRC/GSW1_3/YearlyHistory/2018").clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC_2018,basemap]).sum()
# Maryland
Maryland_2018 = ee.Image('users/311605001111/Maryland/Maryland_2018').clip(roi).select('b1').rename('waterclass')
Maryland_permanent = Maryland_2018.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
Maryland_season= ee.ImageCollection([Maryland_2018.gte(25),Maryland_2018.lt(75)]).sum().eq(2)
Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
# 三者的交集
permanentWaterExtent = ee.ImageCollection([index_waterclass.eq(2),JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(3)
landExtent = ee.ImageCollection([index_waterclass.eq(0),JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(3)
seasonWaterExtent = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')

Map.addLayer(permanentWaterExtent.selfMask(),{'palette':['blue']},"permanent_water")
Map.addLayer(seasonWaterExtent.selfMask(),{'palette':['green']},"season water")

# 封装为函数

In [ ]:
image = sample_image.first()
Map.addLayer(image,visParams,'image')

# print(image.get('system:index').getInfo())

In [ ]:
# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','mNDWI','AWEI']
# 分类标签
label = 'waterclass'
def imageSample(image):
    # 永久性水体
    permanent_points = image.updateMask(permanentWaterExtent).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': 500,
        'seed': 0,
        'geometries': True,
    })
    water_points = permanent_points.map(lambda i : i.setMulti({'waterclass':1,'Image_id':image.get('system:index')}))
    # 陆地
    land_points = image.updateMask(landExtent).sample(**{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': 500,
        'seed': 0,
        'geometries': True,
    })
    nowater_points = land_points.map(lambda i : i.setMulti({'waterclass':0,'Image_id':image.get('system:index')}))
    # 季节性水体
    awei = image.updateMask(seasonWaterExtent).select('AWEI').gt(ee.Image.constant(0))
    seasonWater = image.updateMask(awei).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': 500,
        'seed': 0,
        'geometries': True,
    })
    seasonWater_points = seasonWater.map(lambda i : i.setMulti({'waterclass':1,'Image_id':image.get('system:index')}))
    seasonLand= image.updateMask(awei.remap([0,1],[1,0])).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': 500,
        'seed': 0,
        'geometries': True,
    })
    seasonLand_points = seasonLand.map(lambda i : i.setMulti({'waterclass':0,'Image_id':image.get('system:index')}))
    sample_points = ee.FeatureCollection([water_points,nowater_points,seasonWater_points,seasonLand_points]).flatten()
    return sample_points

In [ ]:
a = imageSample(image)
print(a.size().getInfo())
print(a.aggregate_array('waterclass').getInfo())

# 分步验证

In [ ]:
image = sample_image.first()
Map.addLayer(image,visParams,'image')

In [ ]:
# 永久性水体
permanent = image.updateMask(permanentWaterExtent)
Map.addLayer(permanent,visParams,'permanent image')
permanent_points = permanent.sample(**{
    'region': roi,
    'scale': 30,
    'numPixels': 500,
    'seed': 0,
    'geometries': True,
})
water_points = permanent_points.map(lambda i : i.setMulti({'waterclass':1,'Image_id':image.get('system:index')}))

print(water_points.size().getInfo())
print(water_points.first().getInfo())

In [ ]:
# 陆地水体
land = image.updateMask(landExtent)
Map.addLayer(land,visParams,'land image')
land_points = land.sample(**{
    'region': image.geometry(),
    'scale': 30,
    'numPixels': 500,
    'seed': 0,
    'geometries': True,
})
nowater_points = land_points.map(lambda i : i.setMulti({'waterclass':0,'Image_id':image.get('system:index')}))
print(nowater_points.size().getInfo())
print(nowater_points.first().getInfo())

In [ ]:
# 季节性水体
awei = image.updateMask(seasonWaterExtent).select('AWEI').gt(ee.Image.constant(0))
season_water = image.updateMask(awei)
seasonWater = season_water.sample(**{
    'region': roi,
    'scale': 30,
    'numPixels': 500,
    'seed': 0,
    'geometries': True,
})
seasonWater_points = seasonWater.map(lambda i : i.setMulti({'waterclass':1,'Image_id':image.get('system:index')}))
print(seasonWater_points.size().getInfo())
print(seasonWater_points.first().getInfo())

season_land = image.updateMask(awei.remap([0,1],[1,0]))
seasonLand= season_land.sample(**{
    'region': roi,
    'scale': 30,
    'numPixels': 500,
    'seed': 0,
    'geometries': True,
})
seasonLand_points = seasonLand.map(lambda i : i.setMulti({'waterclass':0,'Image_id':image.get('system:index')}))
print(seasonLand_points.size().getInfo())
print(seasonLand_points.first().getInfo())

In [ ]:
sample_points = ee.FeatureCollection([water_points,nowater_points,seasonWater_points,seasonLand_points]).flatten()
print(sample_points.size().getInfo())
print(sample_points.aggregate_array('waterclass').getInfo())